In [1]:
from functools import partial
from multiprocessing import Pool

import numpy as np
import scipy as sp
import pathpy as pp
from tqdm import tqdm

### Microstates from ensembles

In [2]:
n = 30

We have chosen the [Pathpy3](https://github.com/pathpy/pathpy) package because of the broad list of model ansembles and methods to calculate mean degree and neighbour mean degree. Let's get into the code - this piece of code generate a random network $G(n=50, p=0.1)$ from the $G_{np}$ ensemble and calculate the mean degree and the mean neighbour degree.

In [3]:
network = pp.generators.ER_np(n, 0.1, directed=False, loops=True)
print(f"<k>: {network.mean_degree()}")
print(f"<k_n>: {network.mean_neighbor_degree()}")
print(f"<k_n> - <k>: {network.mean_neighbor_degree() - network.mean_degree()}")

<k>: 2.8333333333333335
<k_n>: 3.3529411764705883
<k_n> - <k>: 0.5196078431372548


We will generate generate for 10 networks per ensemble with some params and calculate mean difference between mean neighbour degrees and mean degrees. First the list of all $G_{np}$ models.

In [4]:
gnps = [
    partial(pp.generators.ER_np, n=n, p=p, directed=False, loops=True)
    for p in np.linspace(0.4, 0.9, 10)
]
gnps

[functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.4, directed=False, loops=True),
 functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.4555555555555556, directed=False, loops=True),
 functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.5111111111111111, directed=False, loops=True),
 functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.5666666666666667, directed=False, loops=True),
 functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.6222222222222222, directed=False, loops=True),
 functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.6777777777777778, directed=False, loops=True),
 functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.7333333333333334, directed=False, loops=True),
 functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.7888888888888889, directed=False, loops=True),
 functools.partial(<function ER_np at 0x7f8811dfdfc0>, n=30, p=0.8444444444444444, directed=False, loops=True),
 functo

Then the $G_{nm}$ ensemble.

In [5]:
gnms = [
    partial(pp.generators.ER_nm, n=n, m=m, directed=False, loops=True, multiedges=False)
    for m in np.linspace(2 * n, pow(n, 2) / 2 - 10, 10, dtype=int)
]
gnms

[functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=60, directed=False, loops=True, multiedges=False),
 functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=102, directed=False, loops=True, multiedges=False),
 functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=144, directed=False, loops=True, multiedges=False),
 functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=186, directed=False, loops=True, multiedges=False),
 functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=228, directed=False, loops=True, multiedges=False),
 functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=271, directed=False, loops=True, multiedges=False),
 functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=313, directed=False, loops=True, multiedges=False),
 functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=355, directed=False, loops=True, multiedges=False),
 functools.partial(<function ER_nm at 0x7f8811dfdea0>, n=30, m=397, directed=Fals

Here is the Watts Strogatz models

In [6]:
wattz_strogatzs = [
    partial(pp.generators.Watts_Strogatz, n=n, s=5, p=p, loops=True)
    for p in np.linspace(0.3, 0.8, 10)
]
wattz_strogatzs

[functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.3, loops=True),
 functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.3555555555555555, loops=True),
 functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.4111111111111111, loops=True),
 functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.4666666666666667, loops=True),
 functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.5222222222222221, loops=True),
 functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.5777777777777777, loops=True),
 functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.6333333333333333, loops=True),
 functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.6888888888888889, loops=True),
 functools.partial(<function Watts_Strogatz at 0x7f8811dfe0e0>, n=30, s=5, p=0.7444444444444445, loops=True),
 functools.partial(<funct

Here will be scale-free networks. They will be quite heterogenious.

In [7]:
def find_graphic_sequence(gamma):
    while True:
        degrees_sequence = sp.stats.zipf.rvs(gamma, size=n)
        if pp.generators.is_graphic_Erdos_Gallai(degrees_sequence):
            return degrees_sequence


sfns = list(
    partial(pp.generators.Molloy_Reed, degrees=degree_seq)
    for gamma in tqdm(np.linspace(2.1, 2.7, 10))
    if (degree_seq := find_graphic_sequence(gamma)) is not None
)
sfns

100%|██████████| 10/10 [00:00<00:00, 2979.97it/s]


[functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=array([ 1,  4,  1,  2,  1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  6,  1,
         1,  1,  1,  1, 20,  2,  1,  1,  1,  1,  1,  2,  2])),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=array([1, 1, 2, 1, 1, 1, 6, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2,
        1, 1, 1, 1, 1, 3, 1, 1])),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=array([ 1,  6,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1, 14,  2,  1,  1,  1,
         1,  1,  2,  2,  1,  1,  1,  1,  1,  2,  4,  7,  1])),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=array([2, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 3, 3, 1, 1, 1, 1,
        1, 1, 3, 1, 1, 2, 1, 1])),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=array([ 1,  1,  4,  1,  1,  1,  3,  1,  1,  1,  1,  1,  1,  1,  1,  7,  1,
         1, 11,  2,  1,  1,  1,  1,  1,  1,  3,  1,  1,  1])),
 functools.partial(<funct

In [8]:
def generator(ensemble, samples=10):
    differences = (
        network.mean_neighbor_degree() - network.mean_degree()
        for _ in range(samples)
        if (network := ensemble()) is not None
    )
    return np.mean(list(differences))

All generated microstates are heterogenuous.

In [9]:
all_models = gnps + gnms + wattz_strogatzs + sfns
len(all_models)

40

Now we will run generation of models on all cpus with multiprocessing.

In [10]:
with Pool() as pool:
    results = np.array(list(tqdm(pool.imap(generator, all_models), total=len(all_models))))

100%|██████████| 40/40 [00:01<00:00, 32.01it/s]


In [11]:
np.any(results == 0)

False

So no of the difference is 0, so mean neighbour degree in theese networks is bigger than mean degree. But now we will try regular networks with different degrees. They are homogenious.

In [12]:
rns = list(
    partial(pp.generators.Molloy_Reed, degrees=degree_seq)
    for i in range(2, 12)
    if (degree_seq := [i] * n) is not None
)
rns

[functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]),
 functools.partial(<function Molloy_Reed at 0x7f8811dfe290>, degrees=[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
 functools.partial(<function

In [13]:
with Pool() as pool:
    homo_results = np.array(list(tqdm(pool.imap(generator, rns), total=len(rns))))

homo_results

100%|██████████| 10/10 [00:00<00:00, 28.60it/s]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

As far as you can judge there is no Friendship Paradox for homogenious networks.

### Real networks
We will take Karate Club, Dolphins and Moreno real networks.

In [14]:
real_networks = [
    pp.io.konect.read_konect_name("moreno_train"),
    pp.io.konect.read_konect_name("ucidata-zachary"),
    pp.io.konect.read_konect_name("dolphins")
]
real_networks

[11-27 10:22:54: WARNING] KONECT data contains invalid timeiso: 1994/2001


In [15]:
differncies = [
    n.mean_neighbor_degree() - n.mean_degree()
    for n in real_networks
]
differncies

[5.0029578189300405, 3.1809954751131224, 1.6759991884763643]

As far as you can judge real networks have even bigger difference between mean neighbour degree and mean degree.